In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import cv2
import csv
import h5py
import random
import matplotlib.pyplot as plt

# Data Preprocessing : Video and Gaze

In [ ]:
#for calculating the timestamps and frame numbers of the images which are extracted. This is just for tracking purpose.

datu = np.load('/bigpool/UbiCompDataCleaned/P01AnnotationCleaned.npy')
abc = datu[:,(0,1)].astype(float)
abc = abc.astype(int)
(r,s) = abc.shape

gdata1 = np.load('/bigpool/UbiCompDataCleaned/P01RecordingCleaned.npy')
(l,m)=gdata1.shape
gdata1[:,1] = gdata1[:,1].astype(int)
u = np.unique(gdata1[:,1])
(ns,) = u.shape
di = np.zeros((ns,2))

e = 0
for i in range(ns):
    print ("curent second is",i, "out of", ns)
    for q in range(e,r):
        if u[i] == abc[q,1]:
            di[i,1] = abc[q,0]
            di[i,0] = abc[q,1]
            e=q
            break
            
np.savez('P1_timestamps_frames.npz', d=d) 

#extracting the frames, resizing them to one fourth of its size and storing as  numpy arrays

%%time
import cv2
print(cv2.__version__)

vid = cv2.VideoCapture("/bigpool/UbiComp2015/Recordings/P01_2014_05_20/4Recording/world_Rec.mkv")
images = []
small_images = []
d = []
for i in range(ns):
    print ("%d out of %d",i,(ns-1))
    vid.set(1,di[i,1]);
    success,image = vid.read() 
    small_image = cv2.resize(image, (0,0), fx=0.25, fy=0.25)  
    small_images.append(small_image)
    print('Read a new frame - number : %d '%di[i,1], success)
small_images_np = np.asarray(small_images)

np.savez('P1_OneSec.npz', small_images_np=small_images_np)

In [ ]:
#loading gaze
gdata1 = np.load('/bigpool/UbiCompDataCleaned/P01RecordingCleaned.npy')
(l,m)=gdata1.shape
gdata1[:,1] = gdata1[:,1].astype(int) 
(len,)=gdata1[:,1].shape
u = np.unique(gdata1[:,1])
(ns,) = u.shape
d = np.zeros((ns,30,7))
e = 0

# converting to 30 array format. 30 gaze data per second
for i in range(ns):
    print ("curent second is",i, "out of", ns)
    ix = 0
    for q in range(e,len):
        if u[i] == gdata1[q,1]:
            if ix < 30 :
                d[i,ix,:] = gdata1[q,(2,3,4,5,7,8,9)]
                #print (ix)
                ix = ix+1
        else:
            #if less than 30 gaze points per sec, then replicate the last value till 30
            while (ix < 30) :
                d[i,ix,:] = d[i,ix-1,:]
                #print (ix)
                ix = ix+1
            e = q            
            break
                
        
            
np.savez('P1_30array.npz', d=d)       

# Data Preprocessing : Label

In [ ]:
#loading the data from datafiles
gdata1 = np.load('/bigpool/UbiCompDataCleaned/P02RecordingCleaned.npy')
adata1 = np.load('/bigpool/UbiCompDataCleaned/P02AnnotationCleaned.npy')

#coping only the timestamps and label encodings into a numpy variable
abc = adata1[:,(1,5,6,7,8,9,10,11,12,13)].astype(float)
#convertion to integers, so that timestamps is represented as seconds
abc = abc.astype(int)
(l,m)=adata1.shape
#copying only the timestamps of the gaze data and converting them to integers to represent in seconds.
gdata1[:,1] = gdata1[:,1].astype(int)
#removing the duplicate values of the timestamps(seconds)
u = np.unique(gdata1[:,1])
#copying the number of unique time stamps
(ns,) = u.shape
#creating an empty array to store the label encoding data in "30 label per second" format
d = np.zeros((ns,30,10))
e = 0
# preparation of "30 label per second" array. For loop runs for all timestamp values (all seconds)
for i in range(ns):
    print ("curent second is",i, "out of", ns)
    ix = 0
    cnt = 0
    for q in range(e,l): 
        #when second's value matches, corresponding first 30 label encoding  are taken
        if u[i] == abc[q,0]:
            cnt = cnt + 1            
            if ix < 30 :                
                d[i,ix,:] = abc[q,(0,1,2,3,4,5,6,7,8,9)]
                ix = ix+1
        else:
            #if there are less than 30 gaze data then last available label encoding is copied to other missing places
            if (cnt != 0 ):
                while (ix < 30) :
                    d[i,ix,:] = d[i,ix-1,:]
                    #print (ix)
                    ix = ix+1
                e = q                
                if (e == (l-1)) and (i != (ns-1)):
                    print (u[i],"second is missing in annotated array")                                
                break
            else:
                dummy = 0                
                

# pre-code to make "single label per second " from "30 label per second "
d_s = np.zeros((30,10))
print("hello")
for i in range(ns):
    print ("curent second is",i, "out of ", ns-1)
    # if all 30 labels per secong is same, no action taken
    d_s = d[i,:,:]
    u_s = np.unique(d_s,axis=0)
    (l,m) = u_s.shape
    # if all 30 labels per second have more than 1 label, then replace the label (which occured majority times) with other ones
    if (l != 1):
        print("More unique values for ",i)
        print(u_s)
        #print(u_s.shape)
        print ("before:", d[i,:,:])
        cnt = np.zeros(l)
        for j in range(l):
            for k in range(30):
                if np.array_equal(u_s[j,:], d_s[k,:]):
                    cnt[j] += 1
        o = np.argmax(cnt)
        print("count is array :",cnt)
        for p in range(30):
            d[i,p,:] = u_s[o,:] 
        print ("after:" , d[i,:,:]) 
        
#making "single label per second" from "30 label per second"
d = d.astype(int)
(l,m,n) = d.shape
label = np.zeros((l,9))
for i in range(l):
    label[i,:] =  d[i, 0, (1,2,3,4,5,6,7,8,9)]        

    
#storing the prepared label in the numpy files
label = label.astype(int)
np.savez('P2_OneSec_OutputLabels.npz', label=label)    